# NegativeBinomial (Benchmark)

http://blog.drivendata.org/2016/12/23/dengue-benchmark/

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from __future__ import division
from __future__ import print_function

from pprint import pprint as pp

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import ParameterSampler
from sklearn.decomposition import PCA


from lib import utils
from lib import cols
from lib import glm_utils

from lib import preprocess


## Preprocessing

In [3]:
ds_sj, ds_iq = preprocess.preprocess()

lib/utils.py:184: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_train_iq = df_dev_iq[is_train_iq]
lib/utils.py:185: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_valid_iq = df_dev_iq[is_valid_iq]
lib/utils.py:186: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_devtest_iq = df_dev_iq[is_devtest_iq]


sj train: 675 lines	 valid: 156 lines	 devtest: 105 lines	 test: 260 lines
iq train: 363 lines	 valid: 104 lines	 devtest: 53 lines	 test: 156 lines
sj valid: 0.19%
iq valid: 0.22%


## Define Model

In [4]:
from statsmodels.tools import eval_measures
import statsmodels.formula.api as smf
import statsmodels.api as sm

base_formula = "total_cases ~ 1 + " \
                    "reanalysis_specific_humidity_g_per_kg + " \
                    "reanalysis_dew_point_temp_k + " \
                    "station_min_temp_c + " \
                    "station_avg_temp_c"

base_cols = ["reanalysis_specific_humidity_g_per_kg", "reanalysis_dew_point_temp_k", "station_min_temp_c", "station_avg_temp_c"]

def get_best_model(train, valid, test, model_formula):
    # Step 1: specify the form of the model
    
    grid = 10 ** np.arange(-8, -3, dtype=np.float64)
                    
    best_alpha = []
    best_valid_score = 1000
        
    # Step 2: Find the best hyper parameter, alpha
    for alpha in grid:
        model = smf.glm(formula=model_formula,
                        data=train,
                        family=sm.families.NegativeBinomial(alpha=alpha))

        results = model.fit()
        predictions = results.predict(valid).astype(int)
        valid_score = eval_measures.meanabs(predictions, valid.total_cases)
        
        test_predictions = results.predict(test).astype(int)
        test_score = eval_measures.meanabs(test_predictions, test.total_cases)

        if valid_score < best_valid_score:
            best_alpha = alpha
            best_valid_score = valid_score
            best_test_score = test_score

    print('best alpha = ', best_alpha)
    print('best valid score = ', best_valid_score)
    print('best test score = ', best_test_score)
            
    # Step 3: refit on entire dataset
    full_dataset = pd.concat([train, valid, test])
    model = smf.glm(formula=model_formula,
                    data=full_dataset,
                    family=sm.families.NegativeBinomial(alpha=best_alpha))

    fitted_model = model.fit()
    return fitted_model


## Evaluate model

In [5]:
model_formula = base_formula
col_feats = base_cols + cols.target
sj_best_model = get_best_model(ds_sj.df_train[col_feats], ds_sj.df_valid[col_feats], ds_sj.df_devtest[col_feats], model_formula)
iq_best_model = get_best_model(ds_iq.df_train[col_feats], ds_iq.df_valid[col_feats], ds_iq.df_devtest[col_feats], model_formula)

best alpha =  1e-08
best valid score =  25.108974359
best test score =  26.6476190476
best alpha =  1e-08
best valid score =  8.74038461538
best test score =  3.90566037736


## Make result

In [6]:
glm_utils.save_result(sj_best_model, iq_best_model, col_feats, col_feats, ds_sj.df_test, ds_iq.df_test, "NegativeBinomial")

## PCA

In [8]:
model_formula = "total_cases ~ 1 + pca_1 + pca_2 + pca_3 + pca_4 + pca_5 + pca_6 + pca_7"
col_pca_feats = cols.get_pca_feats(7)
col_feats = col_pca_feats + cols.target
sj_best_model = get_best_model(ds_sj.df_train[col_feats], ds_sj.df_valid[col_feats], ds_sj.df_devtest[col_feats], model_formula)
iq_best_model = get_best_model(ds_iq.df_train[col_feats], ds_iq.df_valid[col_feats], ds_iq.df_devtest[col_feats], model_formula)

best alpha =  1e-08
best valid score =  21.9615384615
best test score =  24.4952380952
best alpha =  1e-08
best valid score =  9.13461538462
best test score =  3.81132075472


In [ ]:
glm_utils.save_result(sj_best_model, iq_best_model, col_feats, col_feats, ds_sj.df_test, ds_iq.df_test, "NegativeBinomial_PCA")